# Gender Classification of Blog Authors

# 1. Motivation

Our project is Gender Classification of Blog Authors. Two reasons to choose this task.<br>
1. Many NLP models shown in our class enthuses solving a realistic problem with NLP knowledges. <br>
2. The proposed methods are in Mukherjee’s paper(Mukherjee and Liu, EMNLP-2010). Most of existing systems recognize the gender of the authors by analyzing author’s writing style and the word classes. These methods are not accurate when the training data is not large. All these methods requires large datasets because the amount of word classes promotes the accuracy. Our classification model follows Mukherjee’s method because it shows how classifying.

# 2. Approach

&emsp; Our system is mostly following the POS tag sequence method(Mukherjee and Liu, EMNLP-2010). The modifications are in some aspects as below.<br>
&emsp; The entire system mainly invovles two parts: 1) Capturing POS tag sequences from the training data by using mine-POS-pats algorithm. 2) Ranking the selected POS tag sequences which would be used in our chosen classifier(SVM, Logistic Regression...). <br>
&emsp; The following sub-sections are the detailed explaination of the system.

 1) Mukherjee's paper used the POS tagger from one of other paper(Tsuruoka and Tsujii, 2005) which needs extra developing time. 2) We added five extra word classes features. 3) We used TF(Term Frequency) to assign value to the features in Word Class feature<br>
   
### &emsp; 2.1  The general introduction of the system:
 &emsp; In this section, we present our system in a big picture from the raw data(a set of files, each file consists a couple of blogs). We also attached the picture below to better describe the entire processing. <br>
    
![title](report_files/flow_chart.png)
&emsp; The raw data in our experiment is a set of files(19,320 files). Each file is labeled male or female, we would use our implemented system to learn POS tag sequences features from them.
The raw data is first tagged by POS tagger. Mukherjee's paper used the POS tagger from one of other paper(Tsuruoka and Tsujii, 2005) which needs extra developing time. We directly used nltk.pos_tag in our development. Now each file are represented as a set of POS tag sequences. <br>
&emsp; Then those files which are represented as a set of POS tag sequences are the input of the mine-POS-pats algorithm. The mine-POS-pats is basically selected POS tag sequence features based on the two constraints. There is detailed explaination in section 2.2. <br>
&emsp; The selected POS tag sequence features are ranked based on five criterias respectively in the EFS algorithm. Each criteria generates a new set of features and EFS generate a union of the five new set in the end. There is detailed explaination in section 2.3<br>
&emsp; The chosen classifier do the classification for us. Each file is given a score by classifier. The classifier then classify each score into a category.In our case, we have two cases, one is the male, and another is the female. These points then become the model that we used to predict the gender of author of testing files. <br>

    
### &emsp; 2.2  mine-POS-pats algorithm:
&emsp; Below is the pseudocode of mine-POS-pats algorithm. The main job of this algorithm is to select POS tag sequnce features from input corpus D. The features are up to MAX-length-gram(MAX-length is set 7 in our experiment). The way it does is iteratively searching n-gram features that satisfy two constraints that we defined based on n-1 gram features that we chosed in the last iteration. The following paragraphs are the detailed explaination.
![title](report_files/mine-POS-pats.png)
&emsp; The mine-POS-pats algorithm accept 4 parameters: ${D,T,minsup,minadherence}$. ${D}$ is the corpus that has already tagged by POS tagger, so the D is a set of documents and each document is a sequence of POS tags.${T}$ is the tag set contained all POS tags used in the corpus. The next two paragraphs define ${minsup,minadherence}.$<br>
&emsp; We first defined two constant variable ${minsup}$ and ${minadherence}$. ${minsup}$ restricted the minimal number of document that each selected feature must appeared. So in the algorithm, each feature is counted so that it can be compared to the ${minsup}$. We need another more robost measurement: ${minadherence}$. The ${minadherence}$ is the minimal expected adherence strength of POS tags in a sequence. The adherence strength is defined by the function fairSCP. <br>
![title](report_files/fairSCP.png)
&emsp; After defining the above two constant variable, we then defined the MAX-length, which is the maxmium lenght of the each feature that returned from the mine-POS-pats algorithm. In our experiment, we defined the MAX-length to be 7. <br>
&emsp; The mine-POS-pats algorithm first select all unigram features f that f.count/n ${\geq minsup}$ and collect these features in set F. The set F is the set to collect the features that satisfy the constraint: f.count/n ${\geq minsup}$, which is the proportion of the document that certain feature appeared. SP is the set to collect the features that satisfy the constraint: fairSCP(f) ${\geq minadherence}$. The algorithm start its main loop from bigram when k = 2. In each iteration, it first generate k gram feature based on k-1 gram feature already collected.The reason for doing this is that those frequent k gram feature(f.count ${\geq minsup}$) is formed by adding a POS tag from k-1 frequent features.The candidate-gen do the generation. For each selected k-1 gram, candiate-gen form all possible k gram feature, e.g: bigrams: PRP VP , all possible trigram features: PRP VP NP, PRP VP PRP, PRP VP RB,...... . After k gram feature formed, the algorithm then check two constraint mentioned above (${minsup,minadherence}$) unitil k-gram. In the end, it collects the union of unigram, bigram,.., MAX-length-gram feature.<br>
![title](report_files/candidate_gen.png)

### &emsp; 2.3  EFS algorithm:
&emsp; ![title](report_files/EFS.png)
&emsp; Above is the flowchart of EFS algorithm we implemented in our model. We implemented EFS algorithm slightly different from the algorithm in the Mukherjee's paper. <br>
&emsp; The EFS algorithm takes the collected features as input from the mine-POS-pats algorithm and do the ranking based on the five criteria below. The five criteria are the measurement of the relation of a given feature f with respect of the gender.<br>
&emsp; ![title](report_files/five_criteria.png)
&emsp; We then select top x% fetures where x is the threshold we defined. We then choose a classifier to generate a set of scores. We chose Logistic Regression in our optimized version. The classifier used the set of scores generated as the training model and this set of scores are represented as a set of point. There is a clear boundary between scores of male authors and female authors. This will be the model to recognize the gender of the author from test data.<br>
&emsp; ![title](report_files/Logistic_regression.png)

### &emsp; 2.4  Gender Preferential Features:
&emsp;  Gender Preferential Features is based on the observation that male and female have different preference of choosing word when they write text. Femal tend to use more emotionally intensive adverbs and adjectives such as “so”, “terribly”, “awfully”, “dreadful” and “quite”. Female's language is more punctuated with attenuated assertions, apologies, questions, personal orientation and support”. On the contrary, male's language is more independent and direct, so male's choice of words tend to be “strong assertions, aggressive, self-promotion, rhetorical questions, authoritative orientation, challenges and humor”(Corney et al., 2002). We used the table below as the Gender Preferential Features in our model. <br>
![title](report_files/gender_preferencial.png)
### &emsp; 2.4  F-meausre Features:
&emsp; F-measure features measure implicitness of the text. Below is the F-measure score formula. Higher F-measure score means explicitness and lower means explicitness. Female use more  pronouns, verbs, adverbs, and interjections and tend to  get lower F-measure socre, and male use more nouns, adjectives, prepositions, and articles and tend to get higher F-measure score. 
$${F = 0.5 * [(freq.noun + freq.adj + freq.prep +
freq.art) – (freq.pron + freq.verb +
freq.adv + freq.int) + 100] } $$
&emsp; The way we implemented the F-measure formula is not same as the way in the paper(Mukherjee and Liu, EMNLP-2010). We treat each component of the formula(freq.noun) as the feature and wrap into our chosen classifier. The way we defined each component is that:<br>
$$  noun = {NN,NNP,NNPS} $$
$$  adj = {JJR,JJS,JJ} $$ 
$$  prep = {IN}  $$ 
$$  art = {DT} $$ 
$$  pron = {PRP$, PRP} $$ 
$$  verb = {VB, VBD, VBP, VBZ}  $$
$$  adv = {RB, RBR, RBS} $$ 
$$  int = {UH} $$   
&emsp; Because either Mukherjee's paper or Corney's paper do not give a explicit defination of each component, we defined them by our linuistic knowledge.

### &emsp; 2.4  Word Classes Features:
&emsp; Word classes feature is from the observation that if we categorized words into different class as in the table below, male and female have different preference of word classes when they write text(Argamon et al., 2007). We used the classes in the table below and we added five extra word classes features: Games, Fun, Poetic, Swearing, Politics, because we think these classes can differ gender. These classes are not the all classes used by the paper(Argamon et al., 2007) proposed this method. We have tested using all those classes and we found it does not differ so much and it is much slower, so we used the two tables below plus the five extra word classes in our model. 
![title](report_files/word_class_1.png)
![title](report_files/word_class_2.png)

# Data

 &emsp; We have two dataset in our development. One is from the paper (Mukherjee and Liu, EMNLP-2010) that proposed the POS tag sequence feature. This dataset in total has roughly 3000 blog files, 50% blog files written by male and 50% blog files written by female. Because we think this dataset is not large enough, so we found another larger dataset from another paper (J. Schler, 2006) who did the same task. This data contains roughly 19000 blog files and the proportion of files written by male and files written by female is balanced. For the convenience, in this paper we use large dataset and small dataset to denote these two datasets. For each dataset, we split 90% to be the training data and 10% to be the test data. 

# Code

&emsp; We did our project from scracth, so all code in our submission are written by ourselves. We used some built-in available functions in some libraries. We used nltk library and called nltk.pos_tag() and nltk.word_tokenize() when we convert blog into POS tag sequences(in auxi.py blog2S.py). We used sklearn to call different classifier(SVC, SVR, LinearSVC, NuSVR) when we training our model. 

# Experimental Setup

&emsp; The did three experiment. The first experiment is to justify which classifier is the best to use in our EFS algorithm to generate the best accuracy. To answer this question , we tried different classifiers and compare the accuracy. We tried two classifier: Logistic Regression, GussionNb, and Linnear SVC. We test each classifier by setting threshold from 0.05, 0.1, 0.15,...to 1.0, where the threshold means how much proportion of the features we used to classify. <br>
&emsp; The second experiment is tring to answer the question that which criteria used to rank features is the best in terms of accuracy. We carried out this experiment by choosing different criteria in our EFS algorithm and comparing the accuracy of each version.<br>
&emsp; The third experiment is justifying which whether adding the POS tag sequence fetures can increase the accuracy. To justify this, we test each class of features that we implemented including the POS tag sequence features. We then combined all features together and compared the accuracy of each version.<br>
&emsp; To clarify, because in the first experiment, we got the result that the Logistic Regression is the best classifier, so we used it in all other experiment. <br>
&emsp; We also compared our model with other model from other paper(Dwivedi 2017) doing the same task.<br>
&emsp; All of three experiment used the larg dataset, and the comparasion used both large dataset and small dataset.<br>

# Results

![title](report_files/experiment1.png)

&emsp; The highest line in the first plot presents the classification of logistic regression. The accuracy starts 0.7157867495 with the threshold 0.05. The accuracy increases with the larger threshold. The accuracy increases up to 0.74 when the threshold is 0.5 and the accuracy is finally robust around 0.74 after the threshold is larger than 0.6. The LinearSVC classification does not perform as well as the logistic regression. The accuracy starts 0.633747412. The accuracy increases faster than the logistic regression between the treshold 0.05 and 0.4. The accuracy of the LinearSVC at the threshold 0.4 is 0.680952381, and the threshold increases slowly after the threshold 0.4. The GussionNB classification performs the worst compared to the other two classification for any threshold we set.

![title](report_files/experiment2.png)

&emsp; The chart above is the result of experiemtn 2. FM denote F-meausre feature and WC denote Word Class features. The word classe features here combined the Gender Prefrential feature and Word Class feature mentioned before. Using FM alone and using WC alone have roughtly the same accuracy which is about 0.67. Combing FM and WC has increased the accuracy to 0.7 accuracy. Combining POS tag sequence, FM and WC is the best method, which got 0.74 accuracy. 

![title](report_files/experiment3.png)

&emsp; The chart above is the result of experiemtn 3. Each column except the last one refers to the five criteria mentioned before. The accuracy of first three critera are roughly the same, and it is about 0.727. The accuracy of CE is about 0.73 and WET has the accuracy about 0.731. CE and WET are slightly better than IG, MI and CHI. Combing all criteria is about 0.71 accuracy, which is better than using any of five alone.<br>

![title](report_files/comparing_method.png)

&emsp; The chart above is the comparsion between our model and the result from another paper that did the same task using the same dataset(Dwivedi 2017). The blue color denotes the result of small dataset and the orange color denote the result of large dataset. The voting classifier method has roughly 0.75 accuracy in smalldataset and 0.72 in large dataset. The BLSTM method has roughly 0.71 accuracy in small dataset and 0.8 in large dataset. Our model has roughly 0.8 accuracy in the small dataset and 0.74 in the large dataset.<br>

# Analysis of the Results

&emsp; From the experiment 1, it is clear that Linear Regression is the best classifier. We also observed the more features we collected(higher threshold), the more accuracy the model get.<br>
&emsp; From the experiment 2, we found that combing the three classes of features(F-measure, word class and POS tag sequence) is the best method in terms of accuracy.<br>
&emsp; From the experiment 3, we found that IG, MI and CHI have the similar accuracy. Combing five of them is the most accurate.<br>
&emsp; From comparsion between our method and the two method in the paper(Dwivedi 2017) we compared, Our method is ourperform the voting classifier method in small dataset but is worse than the BLST model in the large dataset in terms of accuracy.<br>


# Future Work

&emsp; In our experient, we did not used all available features that we can use in each class of feture. There are more features we can employ. For example, in the word class features, the orginal paper(Argamon et al., 2007) suggested more features than what we used in our experiment. Because the main purpose of this project is to prove how effect the POS tag sequence feature is and we did not have time to test such a large set of features. We only used the features chosen by Mukherjee and Liu in there paper. Increasing the amount of features may increase the accuracy as what we observed in our experiment, but different class fetures may overlap and the accuray may not always increase. This could be the part of the furture work.<br>

# References

J. Schler, M. Koppel, S. Argamon and J. Pennebaker (2006). Effects of Age and Gender on &emsp; Blogging in Proceedings of 2006 AAAI Spring Symposium on Computational Approaches  &emsp; for Analyzing Weblogs.<br>
Corney, M., Vel, O. D., Anderson, A., & Mohay, G. (n.d.). Gender-preferential text mining &emsp; of e-mail discourse. 18th Annual Computer Security Applications Conference, 2002. 
&emsp; Arjun Mukherjee and Bing Liu. "Improving Gender Classification of Blog Authors."<br>
Proceedings of Conference on Empirical Methods in Natural Language Processing (EMNLP-10). &emsp; Oct. 9-11, 2010, MIT, Massachusetts, USA. <br>
Dwivedi, V. P., Singh, D. K., Jha, S., & Ranvijay. (2017). Gender Classification of Blog Authors: With Feature Engineering and Deep Learning using LSTM Networks. 2017 Ninth International Conference on Advanced Computing (ICoAC). <br>

In [1]:
# last cell shows the obtainted highest scores on the two datasets
import gen_classify

print('[Running small dataset(1546*2)]')
gen_classify.genClassTest('m.test', 'f.test')
print('\n\n[Running large dataset(9660*2)]')
print('(this will take a while)')
gen_classify.genClassTest('male', 'female')


# !!!information about the code is in the file "code_README.txt" at root directory

/opt/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


[Running small dataset(1546*2)]


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

0.7870967741935484
0.7838709677419354
0.8122977346278317
0.8090614886731392
0.8349514563106796
0.8122977346278317
0.7928802588996764
0.7766990291262136
0.8252427184466019
0.7734627831715211
average of 10 rounds :  0.8007860945818978


[Running large dataset(9660*2)]
(this will take a while)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

0.724120082815735
0.760351966873706
0.7282608695652174
0.7427536231884058
0.7541407867494824
0.7479296066252588
0.7432712215320911
0.7484472049689441
0.7210144927536232
0.739648033126294
average of 10 rounds :  0.7409937888198759


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
